# USING HARRIS HAWK OPTIMIZATION TO OPTIMIZE AN EXTREME LEARNING    MACHINE (ELM)

### IMPORTING THE NECESSARY LIBRARIES

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

### DEFINING THE ELM

In [2]:
class ELM:
    def __init__(self, n_input_nodes, n_hidden_nodes, n_output_nodes):
        self.n_input_nodes = n_input_nodes
        self.n_hidden_nodes = n_hidden_nodes
        self.n_output_nodes = n_output_nodes
        self.input_weights = np.random.normal(size=[self.n_input_nodes, self.n_hidden_nodes])
        self.biases = np.random.normal(size=[self.n_hidden_nodes])
        self.output_weights = np.random.normal(size=[self.n_hidden_nodes, self.n_output_nodes])

    def predict(self, X):
        H = sigmoid(np.dot(X, self.input_weights) + self.biases)
        return sigmoid(np.dot(H, self.output_weights))



### DEFINING THE HARRIS HAWK OPTIMIZATION

In [3]:
class HHO:
    def __init__(self, max_iter=1000, pop_size=50):
        self.max_iter = max_iter
        self.pop_size = pop_size

    def optimize(self, elm_model, X_train, y_train):
        lb = -1
        ub = 1
        dim = elm_model.n_input_nodes * elm_model.n_hidden_nodes + elm_model.n_hidden_nodes + elm_model.n_hidden_nodes * elm_model.n_output_nodes
        X = np.random.uniform(lb, ub, [self.pop_size, dim])
        fitness = np.zeros([self.pop_size])
        for i in range(self.max_iter):
            
            # Calculate fitness of each individual
            
            for j in range(self.pop_size):
                individual = X[j]
                input_weights = individual[:elm_model.n_input_nodes * elm_model.n_hidden_nodes].reshape([elm_model.n_input_nodes, elm_model.n_hidden_nodes])
                biases = individual[elm_model.n_input_nodes * elm_model.n_hidden_nodes : elm_model.n_input_nodes * elm_model.n_hidden_nodes + elm_model.n_hidden_nodes]
                output_weights = individual[elm_model.n_input_nodes * elm_model.n_hidden_nodes + elm_model.n_hidden_nodes:].reshape([elm_model.n_hidden_nodes, elm_model.n_output_nodes])
                elm_model.input_weights = input_weights
                elm_model.biases = biases
                elm_model.output_weights = output_weights
                y_pred = elm_model.predict(X_train)
                fitness[j] = -np.mean(np.abs(y_pred - y_train))
            
            # Sort population by fitness
            
            sorted_indices = np.argsort(fitness)
            X_sorted = X[sorted_indices]
            fitness_sorted = fitness[sorted_indices]
            
            # Update position of each individual
            
            for j in range(self.pop_size):
                if j < self.pop_size // 2:
                    
                    # Perform exploration
                    
                    r1 = np.random.uniform()
                    r2 = np.random.uniform()
                    A1 = 2 * r1 - 1
                    C1 = 2 * r2
                    D_alpha_Xj_minus_Xi = np.abs(C1 * X_sorted[j] - X_sorted[int(np.floor(self.pop_size / 2))] * A1)
                    X_new_j = X_sorted[j] + D_alpha_Xj_minus_Xi
                else:
                    
                    # Perform exploitation
                    
                    r1 = np.random.uniform()
                    r2 = np.random.uniform()
                    A2 = 2 * r1 - 1
                    C2 = 2 * r2
                    D_alpha_Xbest_minus_Xj = np.abs(C2 * X_sorted[0] - X_sorted[j] * A2)
                    X_new_j = X_sorted[j] + D_alpha_Xbest_minus_Xj
                
                # Clip values to [lb, ub]
                
                X_new_j[X_new_j < lb] = lb
                X_new_j[X_new_j > ub] = ub
                
                # Update position of jth individual in population
                
                X[j] = X_new_j
        
        # Set input weights, biases and output weights of ELM model to best individual
        
        best_individual = X_sorted[0]
        input_weights = best_individual[:elm_model.n_input_nodes * elm_model.n_hidden_nodes].reshape([elm_model.n_input_nodes, elm_model.n_hidden_nodes])
        biases = best_individual[elm_model.n_input_nodes * elm_model.n_hidden_nodes : elm_model.n_input_nodes * elm_model.n_hidden_nodes + elm_model.n_hidden_nodes]
        output_weights = best_individual[elm_model.n_input_nodes * elm_model.n_hidden_nodes + elm_model.n_hidden_nodes:].reshape([elm_model.n_hidden_nodes, elm_model.n_output_nodes])
        elm_model.input_weights = input_weights
        elm_model.biases = biases
        elm_model.output_weights = output_weights
        return elm_model


### SEPARATE MAIN FUNCTION TO CALL THE ABOVE FUNCTIONS

In [4]:
if __name__ == '__main__':
    # Load data
    df = pd.read_csv('kc1.csv')
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values.reshape([-1, 1])
    y = y.astype(int)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    # Split data into training set and test set
    n_samples = X.shape[0]
    n_train_samples = int(n_samples * 0.7)
    indices = np.random.permutation(n_samples)
    X_train = X[indices[:n_train_samples]]
    y_train = y[indices[:n_train_samples]]
    X_test = X[indices[n_train_samples:]]
    y_test = y[indices[n_train_samples:]]
    # Create ELM model
    n_input_nodes = X.shape[1]
    n_hidden_nodes = 10
    n_output_nodes = 1
    elm_model = ELM(n_input_nodes, n_hidden_nodes, n_output_nodes)
    # Train ELM model using HHO optimization
    hho_optimizer = HHO(max_iter=200, pop_size=50)
    optimized_elm_model = hho_optimizer.optimize(elm_model, X_train, y_train)
    # Test ELM model
    y_pred_test = optimized_elm_model.predict(X_test)
    accuracy_test = np.mean(np.round(y_pred_test) == y_test)

### TESTING THE ACCURACY OF THE MODEL

In [5]:
y_pred_test1 = (y_pred_test > 0.75).astype(int)
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_test1)
print(accuracy)

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test1))

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_test1))

0.7203791469194313
Confusion Matrix:
[[404 154]
 [ 23  52]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.72      0.82       558
           1       0.25      0.69      0.37        75

    accuracy                           0.72       633
   macro avg       0.60      0.71      0.60       633
weighted avg       0.86      0.72      0.77       633



In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
y_pred_test2 = scaler.fit_transform(y_pred_test)
y_pred_test2 = (y_pred_test2 > 0.5).astype(int)

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_test2)
print(accuracy)
# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test2))

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_test2))


0.7251184834123223
Confusion Matrix:
[[407 151]
 [ 23  52]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.73      0.82       558
           1       0.26      0.69      0.37        75

    accuracy                           0.73       633
   macro avg       0.60      0.71      0.60       633
weighted avg       0.86      0.73      0.77       633

